In [1]:
import pandas as pd
import pymysql
import random
from sklearn.utils import shuffle


conn=pymysql.connect(host='106.75.133.199',port=6823,user='gml_read',passwd='X7CGsW0HBKsdjsG_Msc;af',db='gml_base_qa',use_unicode=True, charset="utf8")
sql="SELECT a.goods_name,a.category_level2,b.name FROM common_goods_info a LEFT JOIN category_info b ON a.category_level2=b.id "
df_all=pd.read_sql(sql,con=conn)
conn.close()
df_model = df_all.loc[df_all["category_level2"].notna(),["goods_name","name"]]
categories = sorted(set(df_model["name"]))
categories_df = pd.DataFrame({"name":categories,"label":range(len(categories))})
df_model = df_model.merge(categories_df,on="name",how="left")
#sampling
def class_sampling(df,frac_set):
    grouped = df.groupby('name',group_keys=False)
    df_index = grouped.apply(lambda x: x.sample(frac=frac_set,random_state=8)).index.tolist()
    return df_index
train_index = class_sampling(df_model,0.8)
train_set = df_model.loc[train_index].reset_index(drop=True)

dev_test_set = df_model.loc[~df_model.index.isin(train_index)].reset_index(drop=True)
dev_index = class_sampling(dev_test_set,0.5)
dev_set = dev_test_set.loc[dev_index].reset_index(drop=True)

test_set = dev_test_set.loc[~dev_test_set.index.isin(dev_index)].reset_index(drop=True)
#shuffle
train_set = shuffle(train_set,random_state=8)
dev_set = shuffle(dev_set,random_state=8)
test_set = shuffle(test_set,random_state=8)

In [21]:
# with open('/home/test2/TextClass/script/dataset/data/train.txt', 'w', encoding='utf-8') as f:
#     for index, line in train_set.iterrows():
#         f.write(str(line["goods_name"].strip().replace('\n', '').replace('\r', '')) + " +++$+++ " + str(line["label"]) + "\n")

In [22]:
# with open('/home/test2/TextClass/script/dataset/data/dev.txt', 'w', encoding='utf-8') as f:
#     for index, line in dev_set.iterrows():
#         f.write(str(line["goods_name"].strip().replace('\n', '').replace('\r', '')) + " +++$+++ " + str(line["label"]) + "\n")

In [23]:
# with open('/home/test2/TextClass/script/dataset/data/test.txt', 'w', encoding='utf-8') as f:
#     for index, line in test_set.iterrows():
#         f.write(str(line["goods_name"].strip().replace('\n', '').replace('\r', '')) + " +++$+++ " + str(line["label"]) + "\n")

In [16]:
# open('/home/test2/TextClass/script/dataset/data/class.txt', mode='w',encoding='utf-8').write('\n'.join(categories) + '\n')

150

In [5]:
# with open('/home/test2/TextClass/script/dataset/data/id_to_cate.txt', 'w', encoding='utf-8') as f:
#     for index, line in categories_df.iterrows():
#         f.write(str(line["name"].strip().replace('\n', '').replace('\r', '')) + " +++$+++ " + str(line["label"]) + "\n")

In [6]:
test_set.tail()

,goods_name,name,label
2181,美国Nordic Naturals 儿童草莓味DHA鳕鱼油口服液 119ml,宝宝食品,9
10601,DERMACEPT C10铂金抗氧套装,个人洗护,0
2033,理肤泉去痘印修复乳40ml小包装,护理护肤,14
9556,JBL GO2蓝牙音箱海军蓝,家用家电,11
4547,欧乐B(Oral-B)多动向活力按摩牙刷,口腔护理,2
